# Train with custom container

In [31]:
import json
import joblib
import os
import pandas as pd
import time

## Create a training image

### Configure Cloud Build to use Kaniko

In [2]:
!gcloud config set builds/use_kaniko True

Updated property [builds/use_kaniko].


### Create a docker file

In [3]:
APP_FOLDER = '../training_image'
os.makedirs(APP_FOLDER, exist_ok=True)

In [4]:
%%writefile $APP_FOLDER/Dockerfile

FROM gcr.io/jk-demo1/sklearn-cpu:latest
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting ../training_image/Dockerfile


### Create a training script

In [5]:
%%writefile $APP_FOLDER/train.py

import logging
import os
import subprocess
import sys
import joblib
import fire
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/trained_model/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Overwriting ../training_image/train.py


### Build the image

In [6]:
PROJECT_ID = !gcloud config list project --format "value(core.project)"
PROJECT_ID = PROJECT_ID[0]
IMAGE_REPO_NAME="octane-regression-training"
IMAGE_TAG="latest"
IMAGE_URI="gcr.io/{}/{}:{}".format(PROJECT_ID, IMAGE_REPO_NAME, IMAGE_TAG)

!gcloud builds submit --tag $IMAGE_URI $APP_FOLDER

Creating temporary tarball archive of 2 file(s) totalling 1.9 KiB before compression.
Uploading tarball of [../training_image] to [gs://jk-demo1_cloudbuild/source/1566304857.53-947b99a4385c4006bfabc8c48e0fa840.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jk-demo1/builds/0a320474-6647-4f8b-9004-c8dd004be095].
Logs are available at [https://console.cloud.google.com/gcr/builds/0a320474-6647-4f8b-9004-c8dd004be095?project=826865698127].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "0a320474-6647-4f8b-9004-c8dd004be095"

FETCHSOURCE
Fetching storage object: gs://jk-demo1_cloudbuild/source/1566304857.53-947b99a4385c4006bfabc8c48e0fa840.tgz#1566304857965537
Copying gs://jk-demo1_cloudbuild/source/1566304857.53-947b99a4385c4006bfabc8c48e0fa840.tgz#1566304857965537...
/ [1 files][  1.0 KiB/  1.0 KiB]                                                
Operation completed over 1 objects/1.0 KiB.                                      
B

## Submit a training job

In [7]:
JOB_NAME="JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
REGION="us-west1"
SCALE_TIER="BASIC"
JOB_DIR="gs://jk-demo-jobdir/{}".format(JOB_NAME)

DATA_PATH="gs://jk-demo-datasets/gasdata/training.csv"
N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOB_DIR \
--master-image-uri $IMAGE_URI \
--scale-tier $SCALE_TIER \
-- \
--data_path $DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20190820_124126] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20190820_124126

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20190820_124126
jobId: JOB_20190820_124126
state: QUEUED


In [8]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2019-08-20T12:41:27Z'
etag: V3J2D1Nij90=
jobId: JOB_20190820_124126
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://jk-demo-datasets/gasdata/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://jk-demo-jobdir/JOB_20190820_124126
  masterConfig:
    imageUri: gcr.io/jk-demo1/octane-regression-training:latest
  region: us-west1
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20190820_124126?project=jk-demo1

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20190820_124126&project=jk-demo1
INFO	2019-08-20 12:41:26 +0000	service		Validating job requirements...
INFO	2019-08-20 12:41:27 +0000	service		Job creation request has been successfully validated.
INFO	2019-08-20 12:41:27 +0000	service		Job JOB_20190820_124126 is queued.
INFO	2019-08-20 12:41:27 +0000	service		Waiting for job to be provisioned.
IN

## Deploy using AI Platform Prediction

### Create a custom prediction routine

In [34]:
APP_FOLDER = '../predict_app/'
os.makedirs(APP_FOLDER, exist_ok=True)

#### Define a Predictor class

In [52]:
%%writefile $APP_FOLDER/predict.py



class OctaneRegressor(object):
    """A custom prediction routine for Octane regressor"""
    
    def __init__(self, model):
        """Stores the model loaded in from_path"""
        self._model = model
        
    def predict(self, instances, **kwargs):
        """Runs inference"""
    
        inputs = np.asarray(instances)
        outputs = self._model.predict(preprocessed_inputs)
        
        return outputs.tolist()
        
    @classmethod
    def from_path(cls, model_dir):
        """Loads the model from the joblib file"""
        #model_path = os.path.join(model_dir, 'model.joblib')
        #model = joblib.load(model_path)
        model = None
        
        return cls(model)
    

Overwriting ../predict_app//predict.py


#### Configure dependencies

In [54]:
%%writefile $APP_FOLDER/setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['scikit-learn>=0.21.2', 'pandas>=0.25.0']

setup(
    name='custom-predictor',
    description='Custom prediction routine.',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    scripts=['predict.py']
)

Overwriting ../predict_app//setup.py


#### Create a Python source distribution package

In [55]:
%run ../predict_app/setup.py sdist --formats=gztar

running sdist
running egg_info
creating custom_predictor.egg-info
writing top-level names to custom_predictor.egg-info/top_level.txt
writing requirements to custom_predictor.egg-info/requires.txt
writing dependency_links to custom_predictor.egg-info/dependency_links.txt
writing custom_predictor.egg-info/PKG-INFO
writing manifest file 'custom_predictor.egg-info/SOURCES.txt'
reading manifest file 'custom_predictor.egg-info/SOURCES.txt'
writing manifest file 'custom_predictor.egg-info/SOURCES.txt'


running check


creating custom-predictor-0.1
creating custom-predictor-0.1/custom_predictor.egg-info
copying files to custom-predictor-0.1...
copying custom_predictor.egg-info/PKG-INFO -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/SOURCES.txt -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/dependency_links.txt -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/requires.txt -> custom-predictor-0.1/custom_predictor.egg-info
copying custom_predictor.egg-info/top_level.txt -> custom-predictor-0.1/custom_predictor.egg-info
Writing custom-predictor-0.1/setup.cfg
Creating tar archive
removing 'custom-predictor-0.1' (and everything under it)


#### Copy the source distribution package to GCS

In [56]:
DEPLOYMENT_VERSION = "octane_v3"
CUSTOM_PREDICTOR_PATH = "gs://jk-demo-staging/{}/{}".format(DEPLOYMENT_VERSION, "custom-predictor-0.1.tar.gz")

!gsutil cp dist/CustomPredictor-0.1.tar.gz $CUSTOM_PREDICTOR_PATH

Copying file://dist/CustomPredictor-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  554.0 B/  554.0 B]                                                
Operation completed over 1 objects/554.0 B.                                      


### Create a model resource

In [46]:
MODEL_NAME = "gas_octane_regression"
REGION = "us-central1"
LABELS = "task=regression,domain=chemometrics"

!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

ERROR: (gcloud.ai-platform.models.create) Resource in project [jk-demo1] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


### Create a model version

In [58]:
MODEL_PATH = "{}/trained_model/".format(JOB_DIR)

!gcloud beta ai-platform versions create $DEPLOYMENT_VERSION \
--model=$MODEL_NAME \
--origin=$MODEL_PATH \
--runtime-version=1.14 \
--python-version=3.5 \
--package-uris $CUSTOM_PREDICTOR_PATH \
--prediction-class predict.OctaneRegressor

Creating version (this might take a few minutes)......failed.                  
ERROR: (gcloud.beta.ai-platform.versions.create) Create Version failed. Bad model detected with error:  "Failed to load model: User-provided package custom-predictor-0.1.tar.gz failed to install: Command '['python-default', '-m', 'pip', 'install', '--target=/tmp/custom_lib', '--no-cache-dir', '-b', '/tmp/pip_builds', '/tmp/custom_code/custom-predictor-0.1.tar.gz']' returned non-zero exit status 1 (Error code: 0)"


### Test the model
#### Prepare a file with test instances

In [ ]:
TEST_DATASET_PATH = "gs://jk-demo-datasets/gasdata/testing.csv"
INPUT_FILE = 'instances.json'

df_test = pd.read_csv(TEST_DATASET_PATH).drop('octane', axis=1)
with open(INPUT_FILE, "w") as f:
  for index, row in df_test.iterrows():
    f.write(json.dumps(list(row.values)))
    f.write("\n")

#### Run predictions

In [ ]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $VERSION_NAME \
--json-instances $INPUT_FILE

In [19]:
LOCAL_PATH = '/tmp/model.joblib'
GCS_PATH = "{}/trained_model/model.joblib".format(JOB_DIR)

!gsutil cp $GCS_PATH $LOCAL_PATH

Copying gs://jk-demo-jobdir/JOB_20190820_124126/trained_model/model.joblib...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     
